In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE75415"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE75415"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE75415.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE75415.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE75415.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene exrpression profiling of childhood adrenocortical tumors"
!Series_summary	"Pediatric adrenocortical tumors (ACT) are rare and often fatal malignancies; little is known regarding their etiology and biology. To provide additional insight into the nature of ACT, we determined the gene expression profiles of 24 pediatric tumors (five adenomas, 18 carcinomas, and one undetermined) and seven normal adrenal glands. Distinct patterns of gene expression, validated by quantitative real-time PCR and Western blot analysis, were identified that distinguish normal adrenal cortex from tumor. Differences in gene expression were also identified between adrenocortical adenomas and carcinomas. In addition, pediatric adrenocortical carcinomas were found to share similar patterns of gene expression when compared with those published for adult ACT. This study represents the first microarray analysis of childhood ACT. Our findings lay the groundwork for establishin

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression profiling data for adrenocortical tumors
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (adrenocortical cancer), we can use histologic type (row 1)
trait_row = 1

# Age is not available in the sample characteristics
age_row = None

# Gender is available (row 0)
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary: 1 for adrenocortical carcinoma, 0 for adenoma/normal"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'carcinoma' in value:
        return 1  # Positive for cancer
    elif 'adenoma' in value or 'normal' in value:
        return 0  # Negative for cancer
    else:
        return None  # Unknown or undetermined

def convert_age(value):
    """Age conversion function (not used since age is not available)"""
    return None

def convert_gender(value):
    """Convert gender value to binary: 0 for female, 1 for male"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None  # Unknown gender

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save clinical data to CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM1954726': [0.0, 0.0], 'GSM1954727': [0.0, 0.0], 'GSM1954728': [0.0, 0.0], 'GSM1954729': [0.0, 0.0], 'GSM1954730': [0.0, 0.0], 'GSM1954731': [1.0, 1.0], 'GSM1954732': [1.0, 0.0], 'GSM1954733': [1.0, 1.0], 'GSM1954734': [1.0, 0.0], 'GSM1954735': [1.0, 1.0], 'GSM1954736': [1.0, 0.0], 'GSM1954737': [1.0, 0.0], 'GSM1954738': [1.0, 1.0], 'GSM1954739': [1.0, 1.0], 'GSM1954740': [1.0, 0.0], 'GSM1954741': [1.0, 1.0], 'GSM1954742': [1.0, 1.0], 'GSM1954743': [1.0, 1.0], 'GSM1954744': [1.0, 0.0], 'GSM1954745': [1.0, 0.0], 'GSM1954746': [1.0, 0.0], 'GSM1954747': [1.0, 0.0], 'GSM1954748': [1.0, 0.0], 'GSM1954749': [nan, nan], 'GSM1954750': [0.0, nan], 'GSM1954751': [0.0, nan], 'GSM1954752': [0.0, nan], 'GSM1954753': [0.0, nan], 'GSM1954754': [0.0, nan], 'GSM1954755': [0.0, nan], 'GSM1954756': [0.0, nan]}
Clinical data saved to ../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE75415.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers shown in the previous step.
# These identifiers appear to be microarray probe IDs (like '1007_s_at', '1053_at')
# rather than standard human gene symbols (like 'TP53', 'BRCA1', etc.).
# These are likely Affymetrix probe IDs, which need to be mapped to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns needed for mapping
# Based on the annotation preview, 'ID' is the probe identifier and 'Gene Symbol' contains gene symbols
probe_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get a mapping dataframe with the probe ID and gene symbol columns
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the first few rows of the resulting gene expression dataframe
print("First few rows of gene expression data after mapping:")
print(gene_data.head())

# Print the shape of the gene expression dataframe
print(f"Shape of gene expression dataframe: {gene_data.shape}")


First few rows of gene expression data after mapping:
        GSM1954726  GSM1954727  GSM1954728  GSM1954729  GSM1954730  \
Gene                                                                 
A1CF         435.0       344.6       540.8       268.1       435.4   
A2M         4469.0      7595.8      5954.9      6257.1      6142.2   
A4GALT        42.6        61.3       101.8       140.7       142.6   
A4GNT        352.8       220.6       201.6       110.0       181.6   
AAAS         299.7       268.3       481.9       409.7       488.8   

        GSM1954731  GSM1954732  GSM1954733  GSM1954734  GSM1954735  ...  \
Gene                                                                ...   
A1CF         410.7       514.8       279.2       275.3       418.3  ...   
A2M         4218.6      2760.3      3700.1      4408.8      3862.6  ...   
A4GALT        29.3       279.3        77.1        55.4        63.3  ...   
A4GNT        136.6       380.4       194.5       151.6       198.0  ...   
AAAS 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols using NCBI Gene database...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene data shape: {gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_biased}")

# 5. Conduct final quality validation and save cohort information
note = "Dataset containing gene expression profiles of childhood adrenocortical tumors including adenomas and carcinomas."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Normalizing gene symbols using NCBI Gene database...
After normalization, gene data shape: (13542, 31)


Normalized gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE75415.csv
Linked data shape: (31, 13544)


After handling missing values, linked data shape: (30, 13544)
For the feature 'Adrenocortical_Cancer', the least common label is '0.0' with 12 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Adrenocortical_Cancer' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 8 occurrences. This represents 26.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Is trait biased: False


Linked data saved to ../../output/preprocess/Adrenocortical_Cancer/GSE75415.csv
